In [1]:
from __future__ import print_function

from apiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from httplib2 import Http
from oauth2client import file, client, tools
import io

In [2]:
store = file.Storage('token.json')
creds = store.get()

In [3]:
service = build(serviceName='drive', version='v3', http=creds.authorize(Http()))

In [4]:
project_id = service.files().list( q="name contains 'Guia Anuro' " ).execute().get('files')[0].get('id')
project_root_items = service.files().list( q=f"'{project_id}' in parents").execute()

## Get species data table

In [5]:
species_table = list(filter( lambda x: 'tabela_especies' in x.get('name'), project_root_items.get('files') ))[0]

In [6]:
species_table

{'kind': 'drive#file',
 'id': '1acpcDJI3bOSo9BiEwV2gOwBXgfeoV7Hygj-LY0dVuEk',
 'name': 'tabela_especies.xlsx',
 'mimeType': 'application/vnd.google-apps.spreadsheet'}

In [7]:
file_content = service.files().export_media( fileId=species_table.get('id'), mimeType='text/csv').execute()

In [8]:
fname = 'speciesdata.csv'

with open (fname, 'wb') as f:
    f.write(file_content)

## Get species and families articles

In [9]:
articlesFolderId = list(filter( lambda x: x.get('name')=='Artigos', project_root_items.get('files')))[0].get('id')

In [10]:
def getFamiliesFoldersIds():
    results = service.files().list( q=f"'{articlesFolderId}' in parents and mimeType contains 'vnd.google-apps.folder'",
                                  fields='files(id,name)').execute().get('files')
    return  { r['name']: r['id'] for r in results }

def getFamilyDocuments( family, families_folders_ids ):
    family_folder_id = families_folders_ids.get(family)
    results = service.files().list( 
        q=f"'{family_folder_id}' in parents and mimeType contains 'document'",
        fields='files(id,name)').execute().get('files')
    return results

In [11]:
families_folders_ids = getFamiliesFoldersIds()

In [12]:
ids_to_download = [ f.get('id') for f in getFamilyDocuments( 'leptodactylidae', families_folders_ids ) ]

In [13]:
family_folders_ids = getFamiliesFoldersIds()

In [14]:
data = { family: [doc for doc in getFamilyDocuments(family, family_folders_ids)] for family, family_id in family_folders_ids.items()}

In [15]:
# Update species pages
basepath = '../especies/'

for family in data:
    for doc in [ doc for doc in data[family] if doc['name']!='familia' ]:
        docid = doc['id']
        species = doc['name']
        
        request = service.files().export_media(fileId=docid, mimeType='text/plain')
        with open(basepath+f'{species}.md', 'wb') as f:
            f.write(request.execute())

In [16]:
basepath = '../familias/'
for family in data:
    for doc in [doc for doc in data[family] if doc['name']=='familia']:
        docid=doc['id']
        
        request = service.files().export_media(fileId=docid, mimeType='text/plain')
        with open(basepath+f'{family}.md', 'wb') as f:
            f.write(request.execute())